In [ ]:
# Initialize Otter
import otter
grader = otter.Notebook("Lab5.ipynb")

# **ESS 314: Lab 5**
**Earthquake Location**
* This lab was modified by Yiyu Ni to work with Gradescope (niyiyu@uw.edu).

---
we will be using two different methods (triangulation and grid search) to find the location of an earthquake epicenter.

Distance $d$, time $t$, and velocity $v$ are simply related by

$d = vt$. (1)

Therefore, a P-wave traveling for time $t$ at velocity $V_P$ will move across a distance $r$, where $r$ is given by

$𝑟 = 𝑉_Pt$. (2)

![](fig1.png)

The time necessary for the P-wave to travel a distance r can be obtained by dividing equation (2) by $V_P$:

$𝑡 = 𝑟 /𝑉_P$ (3)

![](fig2.png)

If a P-wave is generated from an earthquake (EQ) at the origin time $𝑇_0$ (the time the earthquake starts), the first wiggle on the seismogram will arrive at the observation station labeled “seismometer 1”, at a distance $r_1$ from the earthquake, at the time $T_0$ given by

$T_p^1 = T_0 + r_1/V_P$ (4)

where the superscript 1 indicates the seismometer.

In equation (4), the first term on the right-hand side (RHS) is the start time of the EQ, and the second term on the RHS is the time it takes the P-wave ray to reach the seismometer located at
a distance $r_0$ from the EQ.

Similarly, an S-wave from the same earthquake traveling at a velocity $V_S$ will arrive at seismometer 1 at time $T_S^1$, given by 

$T_s^1 = T_0 + r_1/V_S$ (5)


Since S-waves travel more slowly than P-waves, $T_S^1 > T_P^1$.

Now we have a problem. Equations (4) and (5) both contain the unknown origin time $T_0$ . However, if we subtract equation (4) from equation (5) we eliminate $T_0$ and obtain the following very useful result:

$T_S^1 - T_P^1 = r_1/V_S - t_1/V_P = r_1 (1/V_S - 1/V_P)$ (6)


Equation (6) defines the S-P time (“S minus P time”) for the event as recorded at seismometer 1: $T_{s-p} = T_S^1 - T_P^1$. We will use $T_{s-p}^1$ to represent the S-P time for this event at station 1.

![](fig3.png)


The S-P time depends on the distance from the earthquake to the seismic station and the relative times between the P and S wave, which can be variable based on 3D earth structure.

As part of Lab 2: you calculated P and S wave traveltime curves for an earthquake located 10 km below the surface. Your plot should have looked something like this:


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
r = np.linspace(0,50,51) # km
vp = 5    # km/s, this is 5000 m/s
vs = 3.5  # km/s, this is 3500 m/s
tp = r/vp
ts = r/vs

plt.figure(figsize=(10, 5))
plt.plot(r,tp, 'b-', label='P') 
plt.plot(r,ts, 'r-', label='S')
plt.xlabel('Epicentral Distance (km)')
plt.ylabel('Travel Time to Seismometer (sec)')
plt.title("Earthquake Travel Times")
plt.legend(loc=1)
plt.grid(True)
plt.show()

<!-- BEGIN QUESTION -->

📍**Question 1a.** What happens to the S-P travel time as you get farther away from the epicenter? (1 point)

_Type your answer here, replacing this text._

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

📍**Question 1b**: An earthquake is recorded at two seismic stations, station A and station B. If station A is 10 km from the epicenter (point on Earth’s surface directly above where the earthquake rupture began), and station B is 100 km from the epicenter, which station will exhibit a larger S-P time? (1 point)

_Type your answer here, replacing this text._

<!-- END QUESTION -->

Comparing equations (3) and (6), you may notice that the time separation between the **P-wave arrival** (peak on the seismogram corresponding to the first arrival of a P-wave at the seismometer) and the **S-wave arrival** (peak on the seismogram corresponding to the first arrival of an S-wave at the seismometer) has an effective velocity by which it increases with distance:

$V_{s-p} = \dfrac{1}{1/V_S - 1/V_P}$ . (7)

The quantity  $V_{s-p}$ is called the **effective S-P velocity**.

Therefore, the distance r1 from seismometer 1 to the earthquake can be computed from seismograms (P-wave and S-wave arrival times) as

$r_1 = V_{s-p}T^1_{s-p}$, (8)

where $ T^1_{s-p} $ is measured on a seismogram, but $ V_{s-p} $ is based on knowledge of Earth properties, as values for both $V_P$ and $V_S$ are necessary to make this calculation (eqn 7). In other words, we need to have some idea of the P-wave and S-wave velocities in the Earth.

We are going to download data from the [IRIS DMC](https://ds.iris.edu/ds/nodes/dmc/) data center. The data was collected from the [Pacific Northwest Seismic Network](https://pnsn.org/). We focus on the M 7.8 October 27, 2012 Haida Gwaii earthquake.


In [ ]:
import obspy
from obspy.clients.fdsn import Client
from obspy import UTCDateTime
# create the link between you and the IRIS-DMC
client    = Client('IRIS')
# Origin time of the Nisqually earthquake   
s1 = obspy.UTCDateTime(2001,2,28,18,54,37)       

# collect the information about what data is available
inv = client.get_stations(network="UW",station="*",channel="BHZ",location="*",starttime=s1,endtime=s1+86400)
print(inv)

# create a list of the stations you found
sta=list()
for k in inv:
    for sta1 in k:
        sta.append(sta1.code)
# print the list of the stations we would be using
print("The stations we will be using are:")
print(sta)

Mapping in python requires a few more packages, so we found a map of where these stations are on the IRIS-GMAP tool https://ds.iris.edu/gmap and annotated the stations.

Search using the network/station name below. You can copy paste them. 
- **Network:** UW
- **Station:** GNW,LON,LTY,RWW,SEP,SQM,TTW

![](map_station.png)

In [ ]:
# download data:
tr=obspy.Stream()
# we will update sta to make sure we have all of the stations

# create a list of the stations you found
sta1=list()
# s1 = obspy.UTCDateTime(2012,10,28,3,4,0) # this is the Haida Gwaii M7.8 2012 earthquake.
# s1 = obspy.UTCDateTime(2001,2,28,18,54,37)   
for ista in sta:
    try:
        tr += client.get_waveforms(network="UW",station=ista,channel="*HZ",location="*",starttime=s1,endtime=s1+100)
        sta1.append(tr[-1].stats.station)
        print(tr[-1].stats.station)
        
    except:
        pass
    
# we actually have fewer stations
print(sta1)
sta=sta1

In [ ]:
# we can use matplotlib
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams["figure.figsize"] = (8,6) #bigger size of figures as defaults

####
itr=0 # you can change and make it range from 0 to len(tr)-1
###
# create a time vector of 60 seconds (1 minute), sampled at the sampling rate of the data.
t=np.linspace(0,60,int(60/tr[itr].stats.delta))

# plot the data
plt.figure(figsize=(12,5))
plt.plot(t,tr[itr].data[:len(t)]/np.max(np.abs(tr[itr].data)),'k') # normalized data
plt.title(tr[itr].stats.station)
plt.xlabel('Time in (s)')
plt.grid(True)
# zoom in on specific time
plt.xlim([0,20])
# plt.xlim([20,100])

Now we can plot each of the data one by one. ``tr`` is an **array** of seismic ***streams***. ***Streams*** are basically **Obspy** objects that combine seismic station information and the time series. We can *iterate* through ``tr`` to plot each waveform. We can also zoom in on specific times to focus on different part of the waveforms.

In [ ]:
print("there are "+str(len(tr))+" seismograms from these stations") # here we use "str" to convert an integer (len) into a string

Now we are going to see how the overall waves compare to each others.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams["figure.figsize"] = (18,12)
for ii,itr in enumerate(tr):
    if itr.stats.station == "SEP":continue # this station "clipped", meaning that the shaking was too intense for the instrument
    
    t=np.linspace(0,60,int(60/itr.stats.delta))
    if len(itr.data)<len(t):continue
        
    crap=itr.copy()
    crap = crap.filter('bandpass',freqmin=1,freqmax=10) # you can play with that.
    # plot
    plt.plot(t,crap.data[:len(t)]/np.max(np.abs(crap.data))+ii,'k')
    plt.text(5,ii+0.3,itr.stats.station,fontsize=14,c='r')
plt.grid(True)
plt.xlim([0,100])
# plt.xticks(np.linspace(0,60,60))
plt.xlabel('Time in sec', fontsize=20)



The figure above shows seismograms (data from seismometers) recording ground motion from the same earthquake at 5 different stations. A few things to note about this data:

* The vertical axis on the plot is the amplitude of ground motion. Individual seismograms or traces are scaled differently, and actually normalized.

* The waveforms may look jaggeted and not pure smooth oscillations. This is because this earthquake shaking was really, really strong and even damaged the soil. This was a very strongly felt earthquake with extensive damage.

* There is a high-frequency signal first, that is the P. The lower frequency that arrives next is likely the S wave.





---
## Earthquake Location using triangulation
For the questions below, assume the earthquake is at the surface (z=0 km), so depth is not a factor.

<!-- BEGIN QUESTION -->

📍**Question 2a**:
Annotate the figure above with where you think the P and S waves are on these seismograms. Submit your plot to GradeScope with your assignment named `fig_2a.png`. (5 points, 1 point for each station)

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

📍**Question 2b** Measure the **S-P (differential) times** for each station and list below in text of python variables. There will be 2 times 5 measurements, a total of 10 measurements to find. (2.5 points, 0.5 point for each station)

You can pick an approximate time on the plot, or use an annotation to your previous figure using circles in Preview  or other. Another way is to plot individual traces as shown above and "zoom in" by setting the command ``plt.xlim([t_1,t_2])``, where ``t_1`` and ``t_2`` are times of your choice. You don't need to be 100% accurate.

In [ ]:
# Type your answer here, replacing "..." with your answer

# tp is P-wave travel time, ts is S-wave travel time
# tsp = ts - tp (all units in seconds)

# fill ... for all stations
tp_TTW = ...; ts_TTW = ...; tsp_TTW = ts_TTW - tp_TTW;
tp_SQM = ...; ts_SQM = ...; tsp_SQM = ts_SQM - tp_SQM;
tp_RWW = ...; ts_RWW = ...; tsp_RWW = ts_RWW - tp_RWW;
tp_LTY = ...; ts_LTY = ...; tsp_LTY = ts_LTY - tp_LTY;
tp_GNW = ...; gp_GNW = ...; tsp_GNW = ts_GNW - tp_GNW;

print(" TTW| SQM| RWW| LTY| GNW")
print("%.1f|%.1f|%.1f|%.1f|%.1f" % (tsp_TTW, tsp_SQM, tsp_RWW, tsp_LTY, tsp_GNW))

<!-- END QUESTION -->

📍**Question 2c** Suppose $V_P$ = 6.0 km/s and $V_S$ = 3.47 km/s. These velocities are based on properties of the Earth’s crust. Use equation (7) to compute the effective $V_{s-p}$. Define and put your answer in a float variable called `v_sp`. (1 point)

In [ ]:
# Type your answer here, replacing "..." with your answer
v_sp = ...
v_sp

In [ ]:
grader.check("q2c")

<!-- BEGIN QUESTION -->

📍**Question 2d** Use equation (8), your computed value of $V_{s-p}$, and the S-P times that you measured for each station to populate the r column in the table below, where r is the distance from each station to the EQ that caused the ground motion recorded in the seismograms. (2.5 points, 0.5 for each)

In [ ]:
# Type your answer here, replacing "..." with your answer
r_TTW = ... # for station TTW
r_SQM = ... # for station SQM
r_RWW = ... # for station RWW
r_LTY = ... # for station LTY
r_GNW = ... # for station GNW
print(" TTW  |  SQM  |  RWW  |  LTY  |  GNW")
print("%.3f|%.3f|%.3f|%.3f|%.3f" % (r_TTW, r_SQM, r_RWW, r_LTY, r_GNW))

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

📍**Question 3a:** On the map provided, annotate with circles around each station with a radius corresponding to the $r$ that you computed for that station. The earthquake epicenter is where all the circles (or the most) intersect. Mark this single point with a star. This method of locating the earthquake epicenter is called triangulation. Note that you only have data for some of the stations on the map. Submit your plot to GradeScope with your assignment named `fig_3a.png`. (2 points)

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

📍**Question 3b:** Give at least two reasons why the circles may not exactly cross at a single point. Don't cite poor drawing – think about the myriad sources of error throughout this process. (2 points)

_Type your answer here, replacing this text._

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

📍**Question 3c:** What is the minimum number of S-P times needed to find the epicenter of an earthquake? Explain in words why your answer is correct. (2 points)

_Type your answer here, replacing this text._

<!-- END QUESTION -->

---
## Earthquake Location using grid search algorithm

Now we are going to locate an earthquake in a different way. The difference between the observed travel time, or the travel time observed from seismic data, and the predicted travel time, or the travel time predicted from a model, is called the travel time residual. One method to find the location is to calculate the predicted travel times on a grid and find the grid point with the smallest residual. This is what we will do today.

The predicted travel time from an earthquake at ($x,y$) to the k$^{th}$ station with coordinates ($x_k,y_k$)

$t^p_k = \dfrac{\sqrt{ (x-x_k)^2 + (y-y_k)^2  }}{v}$, (9)

where $v$ is the appropriate seismic velocity. Note that the superscript $p$ here is not a power, but just denotes the predicted travel time.

You can also use the seismogram recorded at the k$^{th}$ station to find the real, or, **observed travel time**. One way to locate an earthquake is to minimize the difference between the travel times predicted from the model, $t^p$, and observed travel times, $t^0$. The expression for the travel time residual at the k$^{th}$ station is

$r_k = t_k^0 - t_k^p$ (10)

where $t_p^0$ is the observed travel time at station k and $t_k^p$ is the predicted travel time at station k.

We are going to minimize the residuals for a whole network of stations using the method of **least squares**. Here we will define the **misfit** of a certain grid point as the sum of the squared residuals of each station

$\epsilon = \sum_{k=1}^n r_k^2$, (11)

where $n$ is the number of stations.

Our goal is to find the grid point with minimum misfit. We will do this by assuming an earthquake location ($x,y$) and calculating the misfit 𝜖 for that location. We do this for many trial locations, and then find the location that fits the data the best.

The table below has the location (x and y coordinates) of 13 stations, as well as the observed P wave travel times at each station.

We are going to read the travel-time data from the CSV file "travel_times.csv" using pandas. **Pandas** is a wonderful open-source python package to handle structured (tables) data. Here, the `Pwave travel-time` field means observed P-wave travel time.

In [ ]:
import pandas as pd

df = pd.read_csv("./travel_times.csv")
print(df)
# et voila!
# this reads the CSV file (a table) into a Pandas "data frame"

In [ ]:
# Pandas data frames are great.
df.head() # shows the top 5 rows

In [ ]:
df.describe() # runs some basic stats on the data in the file.

In [ ]:
df['x'] # how to select the locations.

In [ ]:
np.asarray(df['x']) # convert the data frame column into a numpy array.

📍**Question 4a:** Assume the P wave velocity is 6 km/s. The steps outlined below will help you write a program in Python that uses this information to locate the earthquake using a grid search. Perform your grid search on a 100x100 km grid (10,000 locations). (2 points)


In the following, define variables: i) a P-wave velocity as a single scalar into `vp`, ii) the ``x`` and ``y`` arrays of spatial coordinate, with **values ranging from 1 to 100 with increment of 1**, iii) convert the stations coordinates and travel times from the pandas column into numpy arrays using the function above.

In [ ]:
# Type your answer here, replacing "..." with your answer
vp = ...
x  = ...
y  = ...
print(f"There are {len(x)} points in x and {len(y)} points in y.")

In [ ]:
grader.check("q4a")

<!-- BEGIN QUESTION -->

Now we will initialize an abnormally large value of what residuals could be. We will update this value within a loop to only find the minimum residual among all possible locations.

In [ ]:
resmin = 1e9

📍**Question 4b:** Use nested FOR loops to go through the x and y coordinates of your grid (e.g., outer loop goes through the x coordinates, inner loop goes through the y coordinates). (3 points)

With the for loop where we select ``i`` and ``j`` as possible index for ``x`` and ``y``, you can inquire each potential earthquake source location of x and y. Calculate for each the distance between your trial earthquake location and each source-station distance.  For each potential earthquake location, please calculate:

* Distance between the potential earthquake location and each station
* Predict the travel time between the potential earthquake location and the station for P waves
* Compute the residual ``res`` between the observation and the prediction (that is, the residual)
* Sum all of the residuals over the 13 stations.
* If that residual is less than ``resmin``, update ``resmin=res`` and save the value of the indices (or location as ``i_best`` and ``j_best``.

In [ ]:
# Type your answer here, replacing "..." with your answer
for i in range(len(x)):
    for j in range(len(y)):
        # calculate residual
        res = ...
        if res < resmin:
            resmin = ...
            i_best = ...
            j_best = ...
print(f"The best location is found at i={i_best}, j={j_best}")

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

📍**Question 4c:** The best earthquake location is that found when the residuals are minimized. So where is the earthquake in the ``x`` and ``y`` space? Put your answer in a variable `source_x` and `source_y`. (1 points)

In [ ]:
# Type your answer here, replacing "..." with your answer
source_x = ...
source_y = ...
print(f"The earthquake is most possibly located at x = {source_x}, y = {source_y}")

<!-- END QUESTION -->



## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit. **Please save before exporting!**

These are the lab 5 of ESS 314 Autumn 2023.

In [ ]:
# Save your notebook first, then run this cell to export your submission.
grader.export(pdf=False, run_tests=True)